# K Nearest Neighbours

KNN is a lazy learner classifier which classifies data based on the 'closeness' factor. It is of two types -
    - 1. KNN Classification
    - 2. KNN Regression
    
KNN Algorithm classifies data based on the calculation of Euclidean Distance between the data points and the test instance. A value of 'k' is selected and then the 'k' nearest or closest neighbours are found depending upon the 'k' number of least euclidean distances from the test example. After that the most frequent occuring or 'majority' class is chosen as the classified class for the test instance.

-----
Training phase and prediction phase. 
- Training Phase: No training, only store all training instances.
- Prediction Phase: When testing instances appear, only then Euclidean distances are calculated and model is learned.

-> “Lazy learning”


#### Choosing right value of 'k'

Small k - Captures the fine structure of problem statement, used for smaller data sets, reduced noise between classes

Large k - Ignores(less sesnsitive) the fine structure or noise present in the problem space. Used for larger data sets

- If some of the features are more importnant than the others or contains more noise, in that case, we use the Weighted Euclidean Distance approach

# Psuedo Algorithm

   1•  Determine parameter K. Generally, K = 1 +- sqrt(n); n = total data points. But, you can also perform KNN on smaller data set (validation set) and determine a good value of 'k'

   2• Calculate the distance between the test instance and all the training instances, using one of the Distance methods:
              
              (i).  'Simple Euclidean Distance     on standardized data'
              (ii). 'Weighted Euclidean Distance   on raw data, where weights are based on range of features to bring every 
                                                                feature on same scale, higher range features (10K, 5K) 
                                                                will have smaller weights and lower range 
                                                                ones (1,10) will have higher weights (Mahalanobis distance) 
                                                                called "weights for standaridzation"   
             
             (iii). 'Weighted Euclidean Distance   on standardized data, where weights are based on importance of features
                                                                         more important features will have bigger weight 
                                                                         values while less important features will have
                                                                         smaller weight values, and insignificant features 
                                                                         will have zero weights,
                                                                         called "weights for importance"

   3• Sort the distances and determine K nearest neighbors and gather the labels

   4• Use one of the methods to determine the predicted class/value:
       - 'Simple Majority Voting (KNN)'
       - 'Distance Weighted Voting (WKNN)' 

# KNN Classifier  


## AIM

Predict the Onset of Diabetes (1) or not (0) for a new set of feature values

----

## Data

- This problem is comprised of 768 observations of medical details for Pima indians patents. 

- All patients are women aged 21 or older. All attributes are numeric, and their units vary from attribute to attribute.

- Each record has a class value that indicates whether the patient suffered an onset of diabetes within 5 years of when the measurements were taken Yes(1) or not(0).

- A good prediction accuracy is 70%-76%.

## Imports

In [2]:
import math
import random
import numpy as np
import pandas as pd

import sklearn
from sklearn.utils import shuffle

## Dataset

In [3]:
data = pd.read_csv('Data/data_5_2_pima-indians-diabetes.data.csv')
data.head()

,Level,Iron_content,Calcium_content,Vitamins_deficiency,Beta_nagative,Beta_postive,Blood_workout,Age,Class
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
data.shape

(768, 9)

#### Features

In [5]:
Features = list(data.columns)
Features.remove("Class")

## Standardization (Feature Scaling)

In [6]:
def standarized_data(df):
    
    for f in Features:
        df[f] = (df[f] - df[f].mean())/df[f].std()
        
    return df

In [7]:
df = standarized_data(data)
df.head()

,Level,Iron_content,Calcium_content,Vitamins_deficiency,Beta_nagative,Beta_postive,Blood_workout,Age,Class
0,0.639530,0.847771,0.149543,0.906679,-0.692439,0.203880,0.468187,1.425067,1
1,-0.844335,-1.122665,-0.160441,0.530556,-0.692439,-0.683976,-0.364823,-0.190548,0
2,1.233077,1.942458,-0.263769,-1.287373,-0.692439,-1.102537,0.604004,-0.105515,1
3,-0.844335,-0.997558,-0.160441,0.154433,0.123221,-0.493721,-0.920163,-1.040871,0
4,-1.141108,0.503727,-1.503707,0.906679,0.765337,1.408828,5.481337,-0.020483,1


# Steps....

## Step 1:- Selected a value of 'k'

In [8]:
# Selecting for 3 nearest neighbours
k = int(np.sqrt(df.shape[0]))
print "Selected value of 'k' is", k

Selected value of 'k' is 27


## Step 2:- Prepare the distance functions 

So as to calulate distance between training and testing data points

### Train Test Split

In [9]:
def train_test_split(df, train_size):
    
    df = shuffle(df).reset_index(drop=True)
    
    train_index = int(train_size*len(df)) + 1
    test_index = train_index + 1
    
    train_data = df[0:train_index]
    test_data = df[test_index:]
    print 'Train Data = ', len(train_data), ' Test Data = ', len(test_data)
    
    return train_data, test_data

In [10]:
train_data, test_data = train_test_split(df, 0.80)

Train Data =  615  Test Data =  152


----

##### Example: DEMO For Only 1 (one) 'test' datapoint  vs  all training data points, for visualization purpose

In [11]:
# This is our 1st train data sample
train_data.head(1)

,Level,Iron_content,Calcium_content,Vitamins_deficiency,Beta_nagative,Beta_postive,Blood_workout,Age,Class
0,-1.141108,-0.121808,0.562856,0.65593,-0.232546,1.675184,-1.155579,-0.785774,0


In [12]:
# This is our 1st test data sample
test_data.head(1)

,Level,Iron_content,Calcium_content,Vitamins_deficiency,Beta_nagative,Beta_postive,Blood_workout,Age,Class
616,0.63953,1.285646,-0.367098,-1.287373,-0.692439,-0.9757,-0.886963,1.425067,1


In [13]:
# dist = ROOT{SUM[(Xik - Xjk)^2]} = ROOT{(Xi1-Xj1)^2 + (Xi2-Xj2)^2 + (Xi3-Xj3)^2 + (Xi4-Xj4)... + (Xin-Xjn)^2}; k - no. features
Distance = (train_data.head(1).values[0] - test_data.head(1).values[0])**2
Distance = np.sqrt(Distance.sum())
print "Euclidean Distance between '1 train sample' & '1 test sample' is = ", Distance, 'units'

Euclidean Distance between '1 train sample' & '1 test sample' is =  4.79494588652 units


----

## Functions to compute 'distance' between all training and testing data points

#### Method1: Simple Euclidean Distance (ED)

Assuming all features are equally importnant

In [14]:
def Euclidean_Distance(train_feature_vector, test_feature_vector):
    ED = np.sqrt(((train_feature_vector - test_feature_vector)**2).sum())
    return ED

#### Method2: Weighted Euclidean Distance : Weights for importance (W_ED)
Assuming some features are more importnant than others, hence giving bigger value of weights to important features and smaller value of weights to less important feature and zero value of weights to insignificant ones

- Weighting Function-- 

    Weights found out by Weighting Function (Normalizing correlation values)
                   X - X_min
      X_norm  = ---------------
                 X_max - X_min
              
      i.e.
                 
             Correlation_Value_1 - min(Correlation_Value)
      W1 =  ----------------------------------------------
            max(Correlation_Value) - min(Correlation_Value)
      
     ...-> Maximum value becomes 1 and the min value becomes 0 and others are squished between [0, 1]

In [15]:
# Finding correlation matrix between all features and target 'class'
Correlation = df.corr()['Class']
Correlation = pd.DataFrame(Correlation)

# Removing self correlation of 'class', present at last index
Correlation = Correlation[:-1] 
Correlation['Weights'] = (Correlation.Class - min(Correlation.Class))/(max(Correlation.Class) - min(Correlation.Class))
Correlation

,Class,Weights
Level,0.221898,0.390597
Iron_content,0.466581,1.000000
Calcium_content,0.065068,0.000000
Vitamins_deficiency,0.074752,0.024118
Beta_nagative,0.130548,0.163082
Beta_postive,0.292695,0.566921
Blood_workout,0.173844,0.270915
Age,0.238356,0.431587


- Correlation Matrix _**Assumed**_ weights based on Correlation values and put into a weighting function (i.e. function used here to weight is 'Normalization')

In [16]:
# To calulated Distance weighted Eucledian Distance. Weights are assumed.
def Weighted_Euclidean_Distance(train_feature_vector, test_feature_vector):
    
    W_ED = 0
    
    # Weight_vector = [W1, W2, W3, ..., W8 ] ; Where, W_i = Weight associated with feature_i
    Weight_vector = np.array(Correlation['Weights'])
    
    # W_ED = sqrt[ W1(x2-x1)^2 + W2(x2-x1)^2 + ... + Wn(x2-x1)^2 ]
    W_ED = Weight_vector * (train_feature_vector - test_feature_vector)**2
    W_ED = np.sqrt(W_ED.sum())
    
    return W_ED

## Step 3:- Finding top 'k' neighbours using KNN

1.KNN using:
    - Simple Euclidean Distance
    - Simple Majority Voting

In [17]:
def KNN(train_data, test_data, k):
    
    Y_test_hat = []
    
    for test_feature_vector in test_data[Features].values:
        
        ED_List = []
        
        # # DISTANCE MEASUREMENT - Using Method 1: Simple Euclidean Distance
        #
        # Calculating ED between each test data row and all train data rows
        for train_feature_vector in train_data[Features].values:
            ED_List.append(Euclidean_Distance(train_feature_vector, test_feature_vector))
        
        # Storing calulcated list of ED along with the training data for better understanding 
        train_data_ED = train_data.copy()
        train_data_ED['Euclidean_Distance'] = ED_List
        
        # Sorting the top 'k' rows for k-nearest neighbours based on shortest ED
        train_data_ED = train_data_ED.sort_values(by='Euclidean_Distance').head(k)
        
        # # VOTING PROCESS - Using Method 1: Simple Majority Voting
        #
        # Finding out the majoirty class value present for k-nearest neighbours
        majorityClass = train_data_ED['Class'].mode()[0]
        
        # Appending to final predicted values list
        Y_test_hat.append(majorityClass)
        
    return Y_test_hat

In [18]:
Y_hat_KNN = KNN(train_data, test_data, k)

2.Distance Weighted KNN (WKNN) using:
    - Weighted Euclidean Distance
    - Distance Weighted Voting

![WKNN Formula](Data/knn.png)

- 'A Novel Weighted Voting for K-Nearest' by (P)Jianping Gou

In [19]:
def WKNN(train_data, test_data, k):
    
    Y_test_hat = []
    
    for test_feature_vector in test_data[Features].values:
        WED_List = []
        
        # # DISTANCE MEASUREMENT - Using Method 2: Weighted Euclidean Distance
        #
        # Calculating W_ED between each test data row and all train data rows
        for train_feature_vector in train_data[Features].values:
            WED_List.append(Weighted_Euclidean_Distance(train_feature_vector, test_feature_vector))

        # Storing calulcated list of ED along with the training data for better understanding 
        train_data_WED = train_data.copy()
        train_data_WED['Weighted_Euclidean_Distance'] = WED_List

        # Sorting the top 'k' rows for k-nearest neighbours based on shortest ED
        train_data_WED = train_data_WED.sort_values(by='Weighted_Euclidean_Distance').head(k)

        # # VOTING PROCESS - Using Method 2: Distance Weighted Voting
        # 
        #         d_max - d_i 
        # W_i =  -------------  ;  W_i corresponds to individual weights to each row
        #        d_max - d_min
        #
        # P(1) = Count(1)/Total x SUM(Weights_1s)
        # P(0) = COunt(0)/Total X SUM(Weights_0s)

        # Storing Counts
        Class_0_Size = len(train_data_WED[train_data_WED.Class == 0])
        Class_1_Size = len(train_data_WED[train_data_WED.Class == 1])
        Total_size = k 

        # Calculating W_i
        d_i = train_data_WED['Weighted_Euclidean_Distance']
        train_data_WED['Weights'] = (max(d_i) - d_i)/(max(d_i) - min(d_i))

        # Summing Weights for each class; W(0) = W_1 + W_3 + W_4
        #                                 W(1) = W_2 + W_5 
        Class_0_Weights_SUM = train_data_WED[train_data_WED.Class == 0]['Weights'].sum()
        Class_1_Weights_SUM = train_data_WED[train_data_WED.Class == 1]['Weights'].sum()

        # Calculating Weight * P(Class)
        Weighted_Vote_0 = (1.0*Class_0_Size/Total_size) * Class_0_Weights_SUM
        Weighted_Vote_1 = (1.0*Class_1_Size/Total_size) * Class_1_Weights_SUM

        if Weighted_Vote_0 > Weighted_Vote_1:
            Y_test_hat.append(0)
        else:
            Y_test_hat.append(1)
        
    return Y_test_hat

In [22]:
Y_hat_WKNN = WKNN(train_data, test_data, k)

## Accuracy

In [23]:
score_KNN,  score_WKNN = 0, 0

for y, y_1, y_2 in zip(test_data.Class, Y_hat_KNN, Y_hat_WKNN):
    
    if y == y_1:
        score_KNN +=1
    
    if y == y_2:
        score_WKNN +=1

score_KNN, score_WKNN = 100.0*score_KNN/len(test_data), 100.0*score_WKNN/len(test_data)

print "Accuracy (KNN) = {}%\nAccuracy (WKNN) = {}%".format(round(score_KNN, 2), round(score_WKNN, 2))

Accuracy (KNN) = 74.34%
Accuracy (WKNN) = 75.66%


----

# Using sklearn in-built libraries...

## Imports

In [25]:
import pandas as pd
import numpy as np

from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.cross_validation import cross_val_score, cross_val_predict
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

from sklearn.neighbors import KNeighborsClassifier

## Dataset

In [26]:
df = pd.read_csv('Data/data_5_2_pima-indians-diabetes.data.csv')
df.head()

,Level,Iron_content,Calcium_content,Vitamins_deficiency,Beta_nagative,Beta_postive,Blood_workout,Age,Class
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [27]:
df.shape

(768, 9)

#### Features

In [28]:
Features = list(df.columns)
Features.remove("Class")

## X, Y

In [29]:
X = df[Features]
Y = df['Class']

## Standardisation 

`Most of the machine learning algorithms are based on the calculation of Euclidean distance between data points, which will be affected if the data is not on same scale and the results will be highly skewed. `

`Thus, weightage will be improper and influenced`

 - Example, 5kg and 5000g

    - For a test sample, let's say of 10 kg - 

    - ED (5, 10) = 5

    - ED (5000, 10) = 4990

`Both are theoretically equal and same, but for the model, 5 is closer (as ED is less) while 5000 is quite far, thus will exhibit 5's class.`

In [30]:
Standardization = StandardScaler()

In [31]:
Standardization.fit(X)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [32]:
X = Standardization.transform(X)

In [33]:
X = pd.DataFrame(X, columns = Features)
X.head()

,Level,Iron_content,Calcium_content,Vitamins_deficiency,Beta_nagative,Beta_postive,Blood_workout,Age
0,0.639947,0.848324,0.149641,0.907270,-0.692891,0.204013,0.468492,1.425995
1,-0.844885,-1.123396,-0.160546,0.530902,-0.692891,-0.684422,-0.365061,-0.190672
2,1.233880,1.943724,-0.263941,-1.288212,-0.692891,-1.103255,0.604397,-0.105584
3,-0.844885,-0.998208,-0.160546,0.154533,0.123302,-0.494043,-0.920763,-1.041549
4,-1.141852,0.504055,-1.504687,0.907270,0.765836,1.409746,5.484909,-0.020496


## Randomly - Choosing the a value of 'k'

k = sqrt(n)

where, n is the total number of data points

In [34]:
k = int(np.sqrt(len(X)))
k

27

## Model

In [35]:
KNN = KNeighborsClassifier(n_neighbors = k, weights='uniform')

## Prediction

In [36]:
Y_hat = cross_val_predict(KNN, X, Y, cv=10)

## Accuracy

In [37]:
print 'Accuracy={}\n{}\n{}'.format(accuracy_score(Y,Y_hat)*100.0, confusion_matrix(Y,Y_hat), classification_report(Y,Y_hat))

Accuracy=74.609375
[[446  54]
 [141 127]]
             precision    recall  f1-score   support

          0       0.76      0.89      0.82       500
          1       0.70      0.47      0.57       268

avg / total       0.74      0.75      0.73       768



    - Precision  : Total positives / Total predicted postives    -> Bot's performance
    - Recall     : Total positives / Total actual positives      -> Bot's performance on test data
    
- Model performing is a little baised towards '0'. Recall for '0' is quite high as compared to that of '1' which means, True Negatives (0) are quite higher than true postives (1).


- By confusion matrix it is clear, TN=446 out of total 500 ; TP=127 out of 268

## Improved Accuracy - Using  *10 Fold CV*

#### Average Accuracy Scores for 10 Fold CV ... (i.e. 10 times entire Y_hat computed, calculated score and averaged in the end)

In [38]:
scores = cross_val_score(KNN, X, Y, cv=10, scoring='accuracy').mean()
print 'Average Accuracy Score = ', round(accuracy_score(Y,Y_hat)*100.0,3), '%'

Average Accuracy Score =  74.609 %


----

## Hyperparameter Tuning: GridSearchCV (best vaue of 'k' )

In [39]:
KNN = KNeighborsClassifier()

In [40]:
grid_params = {'n_neighbors': range(1,50), 'weights': ['uniform', 'distance'], 'p': [1,2]}

In [41]:
grid = GridSearchCV(KNN, grid_params, cv=10, scoring='accuracy')

In [42]:
grid.fit(X, Y)

GridSearchCV(cv=10, error_score='raise',
       estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49], 'weights': ['uniform', 'distance'], 'p': [1, 2]},
       pre_dispatch='2*n_jobs', refit=True, scoring='accuracy', verbose=0)

#### Best model 

In [43]:
grid.best_estimator_

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=24, p=1,
           weights='distance')

#### Best Parameters

In [44]:
grid.best_params_

{'n_neighbors': 24, 'p': 1, 'weights': 'distance'}

#### Best Overall Accuracy Score

In [45]:
grid.best_score_

0.7643229166666666

- So the best model will have:-
    - k=24,
    - p=1, 
    - Distance Weighted Eucledian Distance formula.